In [1]:
import pandas as pd
import numpy as np
import re
import requests
import json

from datetime import date, timedelta

In [2]:
# Generate file names for past data

def filedaterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        file_date = start_date + timedelta(n)
        file_date_stamp = re.sub('\-', '', file_date.strftime("%Y-%m-%d"))
        yield "afss_agg-" + file_date_stamp + ".csv"


In [3]:
final_columns = ['eventdate', 'user', 'AppName', 'requestsize',
                 'responsesize']

raw_df = pd.DataFrame(columns=final_columns)
raw_df = raw_df.astype({'requestsize': int, "responsesize": int })

In [4]:
start_date = date(2021, 9, 2)
end_date = date(2021, 10, 5)
for file_name in filedaterange(start_date, end_date):
    # file_date = single_date.strftime("%Y-%m-%d")
    # file_date_stamp = re.sub('\-', '', file_date)
    # file_date_stamp = re.sub('\-', '', single_date.strftime("%Y-%m-%d"))
    print("geting file   " + file_name)
    input_df = pd.read_csv(file_name)
    input_df = input_df[final_columns]
    input_df['eventdate'] = pd.to_datetime(input_df['eventdate'])
    # input_df[['requestsize', 'responsesize']] = input_df[['requestsize', 'responsesize']].apply(pd.to_numeric)
    # print(input_df.dtypes)
    raw_df = raw_df.append(input_df)



geting file   afss_agg-20210902.csv
geting file   afss_agg-20210903.csv
geting file   afss_agg-20210904.csv
geting file   afss_agg-20210905.csv
geting file   afss_agg-20210906.csv
geting file   afss_agg-20210907.csv
geting file   afss_agg-20210908.csv
geting file   afss_agg-20210909.csv
geting file   afss_agg-20210910.csv
geting file   afss_agg-20210911.csv
geting file   afss_agg-20210912.csv
geting file   afss_agg-20210913.csv
geting file   afss_agg-20210914.csv
geting file   afss_agg-20210915.csv
geting file   afss_agg-20210916.csv
geting file   afss_agg-20210917.csv
geting file   afss_agg-20210918.csv
geting file   afss_agg-20210919.csv
geting file   afss_agg-20210920.csv
geting file   afss_agg-20210921.csv
geting file   afss_agg-20210922.csv
geting file   afss_agg-20210923.csv
geting file   afss_agg-20210924.csv
geting file   afss_agg-20210925.csv
geting file   afss_agg-20210926.csv
geting file   afss_agg-20210927.csv
geting file   afss_agg-20210928.csv
geting file   afss_agg-20210

In [5]:
# Read list of approved and streaming apps and create set of apps
apps = pd.read_csv("afss_approved_apps.csv")

set_of_apps = set(apps['AppName'].unique())

In [6]:
def determine_type(appname):
    if appname in set_of_apps:
        return apps[apps['AppName'] == appname]['Disposition'].item()
    else:
        return "Uncategorized"


In [7]:

# Tag each appname with either 'Approved', 'Streaming', 'Blacklist', or 'Uncategorized'
raw_df['type'] = raw_df['AppName'].map(determine_type)

# Remove iCloud from data
raw_df = raw_df[raw_df['AppName'] != 'iCloud']

# Remove General Browsing category
raw_df = raw_df[raw_df['AppName'] != 'generalbrowsing']
raw_df = raw_df[raw_df['AppName'] != 'General Browsing']

# Remove streaming from data
raw_df = raw_df[raw_df['type'] != 'Streaming']

# Remove Video Conferencing solutions (like Zoom) from the data set
raw_df = raw_df[raw_df['type'] != 'VideoConference']


In [10]:
unclassed = raw_df[raw_df['type'] == 'Uncategorized']

In [11]:
classed = raw_df[raw_df['type'] != 'Uncategorized']

In [16]:
unapps = list(unclassed['AppName'].unique())

In [22]:
unclassed = unclassed[['eventdate', 'AppName', 'requestsize', 'responsesize']]

In [39]:
unclassed_dates = unclassed.groupby(['eventdate', 'AppName'])['requestsize', 'responsesize'].sum()

/var/folders/fw/p92hvc2d5tn3kgvhsxq_p5k40000gn/T/ipykernel_88695/1843515495.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  unclassed_dates = unclassed.groupby(['eventdate', 'AppName'], as_index=False)['requestsize', 'responsesize'].sum()


In [38]:
unclassed_dates

AttributeError: 'DataFrame' object has no attribute 'type'

In [ ]:
latest_time = full_raw['eventdate'].max()
earliest_time = full_raw['eventdate'].min()


In [ ]:
latest_time, earliest_time

In [182]:
time_lst

Index(['Day', 'Week', 'Month', 'Quarter', 'Year'], dtype='object', name='agg')

In [156]:
import datetime as dt

In [187]:
end_date = latest_time.date() + dt.timedelta(days=1)
end_time = dt.datetime.combine(end_date, dt.datetime.min.time())
# Set the start time for aggregation to a set number of days before the end time, depending on the aggregation
start_time = end_time - dt.timedelta(days=30)
# Filter the data set to the appropriate time aggregation
ato_t = full_raw[(full_raw['eventdate'] >= start_time) & (full_raw['eventdate'] < end_time)]

In [ ]:
full_raw.memory_usage(deep=True)

In [ ]:
raw_df.memory_usage(deep=True)

In [ ]:
full_raw.memory_usage(deep=True).sum()/1024/1024/1024

In [163]:
agg_df.to_csv("Agg_test_2.csv")

In [221]:
blocks = pd.read_csv("blocks_300k.csv")

In [223]:
blocks.describe(include="all")

,eventdate,event_id,reason,action,appclass,appname,urlcategory,clientIP,deviceName,user,hostname,count
count,300000,3.000000e+05,300000,300000,300000,300000,300000,300000,207538,300000,300000,300000.0
unique,1,NaN,18,1,7,11,45,2871,3040,4198,1211,NaN
top,2/9/21,NaN,Not allowed to browse this category,Blocked,General Browsing,General Browsing,Gambling,192.168.0.19,APN0JG5J1300F8,jahan.behbahany@nike.com,ip-orp.sbtech.info,NaN
freq,300000,NaN,248534,300000,219311,219311,82993,80485,81895,81895,81891,NaN
mean,NaN,6.930000e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
std,NaN,2.905912e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,NaN,6.930000e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25%,NaN,6.930000e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
50%,NaN,6.930000e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
75%,NaN,6.930000e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [225]:
blocks.memory_usage(deep=True)

Index               128
eventdate      18900000
event_id        2400000
reason         28388247
action         19200000
appclass       21415239
appname        21213759
urlcategory    22066509
clientIP       20707488
deviceName     17673722
user           24214012
hostname       22395842
count           2400000
dtype: int64

In [233]:
blocks.memory_usage(deep=True).sum()/1024/1024/1024

0.20579895563423634